## Sub-Themes (Glove 100d)

In [56]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Flatten, Activation, Bidirectional, GRU
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K


import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import spacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, hamming_loss

In [2]:
X_train_Q1 = pd.read_excel('../data/interim/X_train_Q1_clean.xlsx')
X_valid_Q1 = pd.read_excel('../data/interim/X_valid_Q1_clean.xlsx')

y_train_Q1 = pd.read_excel('../data/interim/y_train_Q1.xlsx')
y_valid_Q1 = pd.read_excel('../data/interim/y_valid_Q1.xlsx')

In [7]:
df_train = pd.concat([X_train_Q1, y_train_Q1.iloc[:,12:-1]], axis = 1)
df_train.head()

,Comment,CPD_Improve_new_employee_orientation,CPD_Improve_performance,CPD_Improve_training,CPD_Provide_opportunities_advancement,CPD_other,CB_Ensure_salary_parity_across_gov,CB_Ensure_salary_parity_with_other_orgs,CB_Improve_medical,CB_Increase_salary,...,VMG_Improve_collaboration,VMG_Improve_program,VMG_Pay_attention_to_the_public_interest,VMG_Review_funding_or_budget,VMG_Remove_political_influence,VMG_other,OTH_Other_related,OTH_Positive_comments,OTH_Survey_feedback,OTH_Covid
0,"to be real about diversity, you need to create...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,better communication from the top down,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,more education applicable to my job,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,12:-1]], axis = 1)
df_valid.head()

,Comment,CPD_Improve_new_employee_orientation,CPD_Improve_performance,CPD_Improve_training,CPD_Provide_opportunities_advancement,CPD_other,CB_Ensure_salary_parity_across_gov,CB_Ensure_salary_parity_with_other_orgs,CB_Improve_medical,CB_Increase_salary,...,VMG_Improve_collaboration,VMG_Improve_program,VMG_Pay_attention_to_the_public_interest,VMG_Review_funding_or_budget,VMG_Remove_political_influence,VMG_other,OTH_Other_related,OTH_Positive_comments,OTH_Survey_feedback,OTH_Covid
0,Change management and articulating a clear vis...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"More interaction with Management, not this us ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,We switched to safetyline for field monitoring...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,JCM's used to be treated as the Managers they ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,actively providing the adequate training/knowl...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data_df_train = df_train
data_df_valid = df_valid

In [10]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
# function adapted from `preprocess` function of lab 3 
# of this course (575 Advance Learning Machine)
def preprocess_comments(text, 
                        min_token_len = 2, 
                        irrelevant_pos = ['PRON', 'SPACE', 'PUNCT', 'ADV', 
                                          'ADP', 'CCONJ', 'AUX', 'PRP'],
                        avoid_entities = ['PERSON', 'ORG', 'LOC', 'GPE']):
# note: Didn't use the following options in the `preprocess_comments`...
#    - 'PROPN' because it erases proper names as 'George', but also words as orange.
#    - 'DET' since it removes the word 'no', which changes the meaning of a sentence.
# *for more information see link: https://universaldependencies.org/u/pos/
    """
    Given text, min_token_len, irrelevant_pos and avoid_entities, carries out 
    preprocessing of the text and returns list of preprocessed text. 
    Parameters
    -------------
    text : (list) 
        the list of text to be preprocessed
    min_token_len : (int) 
        min_token_length required
    irrelevant_pos : (list) 
        a list of irrelevant pos tags
    avoid_entities : (list)
        a list of entity labels to be avoided
    Returns
    -------------
    (list) list of preprocessed text
    Example
    -------------
    >>> example = ["Hello, I'm George and I love swimming!",
                   "I am a really good cook; what about you?",
                   "Contact me at george23@gmail.com"]
    >>> preprocess(example)
    (output:) ['hello love swimming', 'good cook', 'contact']
    """
    result = []
    others = ["'s", "the", "that", "this", "to", "-PRON-"]
    # I add "-PRON-" that erase "my", "your", etc. other way to erase them is to
    #   use adding 'DET' to irrelevant_pos but it would erase the word 'no' too.
    for sent in text:
        sent = sent.lower()
        sent = re.sub(r"facebook", "social media", sent)
        sent = re.sub(r"twitter", "social media", sent)
        sent = re.sub(r"instagram", "social media", sent)
        sent = re.sub(r"whatsapp", "social media", sent)
        sent = re.sub(r"linkedin", "social media", sent)
        sent = re.sub(r"snapchat", "social media", sent)
        result_sent = []
        doc = nlp(sent)
        entities = [str(ent) for ent in doc.ents if ent.label_ in avoid_entities]
        # This helps to detect names of persons, organization and dates
        for token in doc:            
#             if (token.is_stop == True or
            if(token.like_email or
                token.like_url or
                token.pos_ in irrelevant_pos or
                str(token) in entities or
                str(token.lemma_) in others or
                len(token) < min_token_len):
                continue
            else:
                result_sent.append(token.lemma_)
        result.append(" ".join(result_sent))
    return result

## Training & Validation Data

In [21]:
X_train = pd.DataFrame(preprocess_comments(data_df_train['Comment']), columns=['Comment'])
y_train = np.asarray(data_df_train.drop(columns = ['Comment']))

In [22]:
X_valid = pd.DataFrame(preprocess_comments(data_df_valid['Comment']), columns = ['Comment'])
y_valid = np.asarray(data_df_valid.drop(columns = ['Comment']))

## Max length

In [13]:
def max_len(x):
    a=x.split()
    return len(a)

max_len = max(data_df_train['Comment'].apply(max_len))

max_len

150

## Vocab

In [14]:
vect=Tokenizer()
vect.fit_on_texts(data_df_train['Comment'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

12829


## Getting Embeddings

In [15]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/Users/karan/Downloads/glove/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [16]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in vect.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [17]:
embedding_matrix.shape

(12829, 100)

In [18]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.18970001,  0.050024  ,  0.19084001, ..., -0.39804   ,
         0.47646999, -0.15983   ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.53812999,  0.72706997,  0.074018  , ..., -0.41005999,
         1.08850002,  0.75314999],
       [-1.51540005,  0.66566002,  0.23134001, ...,  0.47402   ,
         0.84129   ,  0.94787002]])

## Padding Data

In [24]:
encoded_docs_train = vect.texts_to_sequences(X_train['Comment'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_len, padding='post')
print(padded_docs_train)

[[  611   695    48 ...     0     0     0]
 [  217   114  2027 ...     0     0     0]
 [  106    75   331 ...     0     0     0]
 ...
 [  811    27    45 ...     0     0     0]
 [  581  3127 12827 ...     0     0     0]
 [ 1274   701  2303 ...     0     0     0]]


In [25]:
padded_docs_train.shape

(10376, 150)

In [26]:
encoded_docs_valid = vect.texts_to_sequences(X_valid['Comment'])
max_length = vocab_size
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')
print(padded_docs_valid)

[[ 127   52 2931 ...    0    0    0]
 [  15 1930   52 ...    0    0    0]
 [2457 6774  421 ...    0    0    0]
 ...
 [ 148    9  136 ...    0    0    0]
 [ 322 1139  663 ...    0    0    0]
 [  78  802 2470 ...    0    0    0]]


In [27]:
padded_docs_valid.shape

(2594, 150)

## Model

In [36]:
max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2
embed_size = 100 # for glove we are using 100d dataset
n_class = 62

In [37]:
model = Sequential()

model.add(Embedding(max_features, embed_size, weights=[embedding_matrix],
                        trainable=False, input_length=maxlen))

model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu',
                 strides=1))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(n_class, activation = 'sigmoid'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 150, 100)          1282900   
_________________________________________________________________
dropout_5 (Dropout)          (None, 150, 100)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 148, 250)          75250     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 74, 250)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 72, 250)           187750    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 36, 250)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9000)              0         
__________

In [38]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_docs_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_data=(padded_docs_valid, y_valid))

Train on 10376 samples, validate on 2594 samples
Epoch 1/2
10368/10376 [============================>.] - ETA: 0s - loss: 0.1635 - acc: 0.9598 

/Users/karan/anaconda3/lib/python3.7/site-packages/numpy/ctypeslib.py:436: RuntimeWarning: Invalid PEP 3118 format string: '&<f'
  return array(obj, copy=False)


10376/10376 [==============================] - 1268s 122ms/step - loss: 0.1634 - acc: 0.9598 - val_loss: 0.1108 - val_acc: 0.9749
Epoch 2/2
10376/10376 [==============================] - 1294s 125ms/step - loss: 0.1158 - acc: 0.9750 - val_loss: 0.1028 - val_acc: 0.9754


In [40]:
from sklearn.metrics import precision_score, recall_score, f1_score

pred = model.predict(padded_docs_valid, batch_size=batch_size, verbose=1)
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print('\n')

2594/2594 [==============================] - 240s 93ms/step
For threshold:  0.1
Micro-average quality numbers
Precision: 0.2816, Recall: 0.1203, F1-measure: 0.1686


For threshold:  0.2
Micro-average quality numbers
Precision: 0.6172, Recall: 0.0587, F1-measure: 0.1071


For threshold:  0.3
Micro-average quality numbers
Precision: 0.6747, Recall: 0.0483, F1-measure: 0.0901


For threshold:  0.4
Micro-average quality numbers
Precision: 0.7229, Recall: 0.0413, F1-measure: 0.0782


For threshold:  0.5
Micro-average quality numbers
Precision: 0.7543, Recall: 0.0327, F1-measure: 0.0626


For threshold:  0.6
Micro-average quality numbers
Precision: 0.8087, Recall: 0.0230, F1-measure: 0.0448


For threshold:  0.7
Micro-average quality numbers
Precision: 0.8545, Recall: 0.0116, F1-measure: 0.0230


For threshold:  0.8
Micro-average quality numbers
Precision: 0.9583, Recall: 0.0057, F1-measure: 0.0113


For threshold:  0.9
Micro-average quality numbers
Precision: 1.0000, Recall: 0.0005, F1-meas

## Weighted Cross Entropy for all Sub-Themes

In [41]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfb

POS_WEIGHT =  7  # multiplier for positive targets, needs to be tuned

def weighted_binary_crossentropy(target, output):
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.math.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(labels=target,
                                                    logits=output,
                                                    pos_weight=POS_WEIGHT)
    return tf.reduce_mean(loss, axis=-1)

In [42]:
max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2
embed_size = 100 # for glove we are using 100d dataset
n_class = 62

In [43]:
model = Sequential()

model.add(Embedding(max_features, embed_size, weights=[embedding_matrix],
                        trainable=False, input_length=maxlen))

model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu',
                 strides=1))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(n_class, activation = 'sigmoid'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 150, 100)          1282900   
_________________________________________________________________
dropout_7 (Dropout)          (None, 150, 100)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 148, 250)          75250     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 74, 250)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 72, 250)           187750    
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 36, 250)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 9000)              0         
__________

In [44]:
model.compile(loss=weighted_binary_crossentropy, optimizer='adam', metrics=['accuracy'])

# Train Model
history = model.fit(padded_docs_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(padded_docs_valid, y_valid))

AttributeError: 'Value' object has no attribute 'dtype'

## USE

In [129]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
hub_layer = hub.KerasLayer(model, output_shape=[512], input_shape=[], dtype=tf.string, trainable=True)

In [121]:
inputs2 = tf.keras.layers.Reshape(input_shape=(512,), target_shape=(512, 1))(hub_layer)

#embedding1 = Embedding(max_features, 300, weights=[embedding_matrix_fastext], trainable=False)(inputs1)



bi_gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(278, return_sequences=True))(inputs2) ## changed from 125
global_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_gru)
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_gru)
concat_layer = Concatenate()([global_pool, avg_pool])
output = tf.keras.Dense(n_class, activation='sigmoid')(concat_layer)
model=Model(inputs1, output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_accuracy'])
model.summary()

return model

In [132]:
input = tf.keras.layers.Input(shape=(), name="Input", dtype=tf.string)
x = hub_layer(input)
x = tf.keras.layers.Reshape(input_shape=(512,), target_shape=(512, 1))(x)
bi_gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(278, return_sequences=True))(x) ## changed from 125
global_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_gru)
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_gru)
concat_layer = tf.keras.layers.Concatenate()([global_pool, avg_pool])
output = tf.keras.layers.Dense(n_class, activation='sigmoid')(concat_layer)
model = tf.keras.models.Model(input, output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_9 (KerasLayer)      (None, 512)          256797824   Input[0][0]                      
__________________________________________________________________________________________________
reshape_15 (Reshape)            (None, 512, 1)       0           keras_layer_9[1][0]              
__________________________________________________________________________________________________
bidirectional_14 (Bidirectional (None, 512, 556)     468708      reshape_15[0][0]                 
____________________________________________________________________________________________

In [ ]:
input = Input(shape=(), name="Input", dtype=tf.string)
x = hub_layer(input)
x = tf.keras.layers.Reshape(input_shape=(512,), target_shape=(512, 1))(x)
x = tf.keras.layers.Conv1D(128, 2, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling1D(5, padding='same')(x)
x = tf.keras.layers.Conv1D(128, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling1D(5, padding='same')(x)
x = tf.keras.layers.Conv1D(128, 4, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling1D(40, padding='same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
m = Model(input, output)
m.summary()

In [ ]:
def define_model(length, max_features):
  # channel 1
  inputs1 = Input(shape=(length,))
  embedding1 = Embedding(max_features, 300, weights=[embedding_matrix_fastext], trainable=False)(inputs1)
  # dropout=SpatialDropout1D(0.2)(embedding1)
  bi_gru = Bidirectional(GRU(278, return_sequences=True))(embedding1) ## changed from 125
  # bi_gru2 = Bidirectional(GRU(50, return_sequences=True))(bi_gru)
  # bi_gru3 = Bidirectional(GRU(50))(bi_gru)
  # cnn = Conv1D(filters=120, kernel_size=10, strides=1, padding='valid')(bi_gru2)
  global_pool = GlobalMaxPooling1D()(bi_gru)
  avg_pool = GlobalAveragePooling1D()(bi_gru)
  concat_layer = Concatenate()([global_pool, avg_pool])
  output = Dense(n_class, activation='sigmoid')(concat_layer)
  model=Model(inputs1, output)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_accuracy'])
  model.summary()
  return model

In [133]:
X_train_array = np.asarray(X_train['Comment'])
y_train_array = np.asarray(y_train)

X_valid_array = np.asarray(X_valid['Comment'])
y_valid_array = np.asarray(y_valid)

In [134]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [135]:
history = model.fit(X_train_array,
                    y_train_array,
                    epochs=5,
                    batch_size=512,
                    validation_data=(X_valid_array, y_valid_array),
                    verbose=1)

Train on 10376 samples, validate on 2594 samples
Epoch 1/5
10376/10376 [==============================] - 346s 33ms/sample - loss: 0.4250 - accuracy: 0.9517 - val_loss: 0.1571 - val_accuracy: 0.9749
Epoch 2/5
10376/10376 [==============================] - 323s 31ms/sample - loss: 0.1236 - accuracy: 0.9749 - val_loss: 0.1148 - val_accuracy: 0.9749
Epoch 3/5
10376/10376 [==============================] - 381s 37ms/sample - loss: 0.1122 - accuracy: 0.9749 - val_loss: 0.1107 - val_accuracy: 0.9749
Epoch 4/5
10376/10376 [==============================] - 361s 35ms/sample - loss: 0.1108 - accuracy: 0.9749 - val_loss: 0.1106 - val_accuracy: 0.9749
Epoch 5/5
10376/10376 [==============================] - 312s 30ms/sample - loss: 0.1108 - accuracy: 0.9749 - val_loss: 0.1107 - val_accuracy: 0.9749


In [136]:
pred = model.predict(X_valid_array, batch_size=512, verbose=1)
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid_array, pred, average='micro')
    recall = recall_score(y_valid_array, pred, average='micro')
    f1 = f1_score(y_valid_array, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

2594/2594 [==============================] - 12s 4ms/sample
For threshold:  0.1
Micro-average quality numbers
Precision: 0.1149, Recall: 0.0738, F1-measure: 0.0898
For threshold:  0.2
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.3
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.4
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.5


/Users/karan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.6
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.7
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [57]:
model = tf.keras.Sequential()

inputs1 = hub_layer
#embedding1 = Embedding(max_features, 300, weights=[embedding_matrix_fastext], trainable=False)(inputs1)
bi_gru = Bidirectional(GRU(278, return_sequences=True))(inputs1) ## changed from 125
global_pool = GlobalMaxPooling1D()(bi_gru)
avg_pool = GlobalAveragePooling1D()(bi_gru)
concat_layer = Concatenate()([global_pool, avg_pool])
output = Dense(n_class, activation='sigmoid')(concat_layer)
model=Model(inputs1, output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_accuracy'])
model.summary()

model.summary()

ValueError: Layer bidirectional_1 was called with an input that isn't a symbolic tensor. Received type: <class 'tensorflow_hub.keras_layer.KerasLayer'>. Full input: [<tensorflow_hub.keras_layer.KerasLayer object at 0x7fdc70789e90>]. All inputs to the layer should be tensors.

In [81]:
model = tf.keras.Sequential()

x = (hub_layer)
x = model.add(tf.keras.layers.Reshape(input_shape=(512,), target_shape=(512, 1)))(x)
x = model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(278, return_sequences=True)))(x)

y_1 = model.add(tf.keras.layers.GlobalMaxPool1D())(x)
y_2 = model.add(tf.keras.layers.GlobalAveragePooling1D())(x)

concat_layer = Concatenate()([y_1, y_2])

model.add(tf.keras.layers.Dense(n_class, activation = 'sigmoid'))

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 512)               256797824 
_________________________________________________________________
reshape_3 (Reshape)          (None, 512, 1)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512, 556)          468708    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 556)               0         
_________________________________________________________________
dense (Dense)                (None, 62)                34534     
Total params: 257,301,066
Trainable params: 257,301,066
Non-trainable params: 0
_________________________________________________________________


In [82]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [84]:
X_train_array = np.asarray(X_train['Comment'])
y_train_array = np.asarray(y_train)

X_valid_array = np.asarray(X_valid['Comment'])
y_valid_array = np.asarray(y_valid)

In [85]:
history = model.fit(X_train_array,
                    y_train_array,
                    epochs=5,
                    batch_size=512,
                    validation_data=(X_valid_array, y_valid_array),
                    verbose=1)

Train on 10376 samples, validate on 2594 samples
10376/10376 [==============================] - 344s 33ms/sample - loss: 0.4265 - accuracy: 0.9497 - val_loss: 0.1342 - val_accuracy: 0.9749


In [86]:
pred = model.predict(X_valid_array, batch_size=512, verbose=1)
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid_array, pred, average='micro')
    recall = recall_score(y_valid_array, pred, average='micro')
    f1 = f1_score(y_valid_array, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

2594/2594 [==============================] - 11s 4ms/sample
For threshold:  0.1
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.2
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.3
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.4


/Users/karan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.5
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.6
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.7
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
